In [1]:
from dataframe import Data
from io_utils import *
import matplotlib.pyplot as plt

In [2]:
from pandas import DataFrame

In [3]:
from prediction_evaluation import PredictionEvaluation

In [4]:
train_ob = load_ztf_train_data(head_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_HEAD.FITS', phot_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_master_PHOT.FITS')
test_ob = load_ztf_test_data(head_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_HEAD.FITS', phot_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_master_PHOT.FITS')

#train_ob = load_ztf_train_data()
#test_ob = load_ztf_test_data()

In [5]:
num_pc_components = 3

In [6]:
prediction_type_nos= [150,151]

In [7]:
color_band_dict = {'g':'C2', 'r':'C3'}

In [8]:
sample_numbers_train = {101:150,
               102:150, 
               103:150, 
               112:150, 
               113:150, 
               114:150, 
               141:150, 
               143:150, 
               145:150, 
               150:500, 
               151:500, 
               160:150, 
               161:150, 
               162:150, 
               163:150, 
               164:150, 
               170:150, 
               180:150, 
               181:150, 
               183:150, 
               190:150}
sample_numbers_train=None

In [9]:
sample_numbers_test = {101:150,
               102:150, 
               103:150, 
               112:150, 
               113:150, 
               114:150, 
               141:150, 
               143:150, 
               145:150, 
               150:500, 
               151:500, 
               160:150, 
               161:150, 
               162:150, 
               163:150, 
               164:150, 
               170:150, 
               180:150, 
               181:150, 
               183:150, 
               190:150}
sample_numbers_test=None

In [10]:
train_features_path = '/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv'
test_features_path='/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band_random_current_date_100_days_alert.csv'

In [ ]:
#train_features_path = None
test_features_path = None

In [ ]:
use_random_current_date = True

In [11]:
#catch error if data_ob and path are not compatible 
train_ob.create_features_df(prediction_type_nos=prediction_type_nos, 
                            band_choice='u', 
                            num_pc_components=num_pc_components, 
                            #plot_all_predictions=True,
                            color_band_dict=color_band_dict,
                            sample_numbers=sample_numbers_train, 
                            features_path=train_features_path, 
                            use_random_current_date=False)

In [12]:
from sklearn.ensemble import RandomForestClassifier

bands = ['g', 'r']

cols = []
for i,band in enumerate(bands):
    for j in range(1,num_pc_components+1):
        col_name = str(i)+'pc'+str(j)
        cols.append(col_name)
        
x_train = train_ob.features_df[cols].values
y_train = train_ob.features_df['y_true'].values


clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [13]:
test_ob.create_features_df(color_band_dict=color_band_dict, min_flux_threshold=200,
                           #save_fig_path = 'ZTF_partial_lt_curve/', 
                           prediction_type_nos=prediction_type_nos, 
                           band_choice='u',num_alert_days=30,
                           classifier = clf,
                           #plot_predicted_curve_of_type = [150,151],
                           features_path = test_features_path,
                           num_pc_components=num_pc_components, 
                           sample_numbers=sample_numbers_test, 
                           use_random_current_date=True)

In [14]:
print(test_ob.features_df)

       0n         0pc1         0pc2        0pc3  1n         1pc1         1pc2  \
0       0     0.000000     0.000000    0.000000   0     0.000000     0.000000   
1       0     0.000000     0.000000    0.000000   0     0.000000     0.000000   
2       8  2025.542058  -171.041073  424.917924  10  1172.235061  -904.642394   
3       0     0.000000     0.000000    0.000000   0     0.000000     0.000000   
4       0     0.000000     0.000000    0.000000   0     0.000000     0.000000   
...    ..          ...          ...         ...  ..          ...          ...   
20783  12  1293.879442   364.327012    6.784719  11   575.875096  -275.399232   
20784   9  1762.177621 -2491.134575  153.648109  10   892.235592 -4277.707696   
20785   9   427.893747    58.991295  142.814898   0     0.000000     0.000000   
20786   0     0.000000     0.000000    0.000000   0     0.000000     0.000000   
20787   0     0.000000     0.000000    0.000000   0     0.000000     0.000000   

              1pc3  curr_da

In [ ]:
test_ob.discard_no_featues_events()

In [ ]:
print(test_ob.features_df)

In [ ]:
x_test = test_ob.features_df[cols].values
y_test = test_ob.features_df['y_true'].values

y_pred_train=clf.predict(x_train)
y_pred_test=clf.predict(x_test)
y_score= clf.predict_proba(x_test)

In [ ]:
print(len(y_pred_test))

In [ ]:
print((test_ob.features_df.keys()))

In [ ]:
train_save_df = train_ob.features_df.drop(['y_pred','y_true'],axis=1)
test_save_df = test_ob.features_df.drop(['y_pred','y_true'],axis=1)

train_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv', index=False)
test_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band_random_current_date_30_days_alert.csv', index=False)

In [ ]:
#train_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/train_features_master_3_pcs_u_band.csv', index=False)
#test_save_df.to_csv('/media/biswajit/drive/Kilonova_datasets/ZTF_20190512/test_features_master_3_pcs_u_band.csv', index=False)

In [ ]:
print(clf.predict([[0,0,0,0,0,0]]))

In [ ]:
pe = PredictionEvaluation(train_ob=train_ob, test_ob=test_ob,y_pred_train=y_pred_train, y_pred_test=y_pred_test)

In [17]:
print(test_ob.sample_numbers)

{160: 1000, 161: 1000, 162: 1000, 163: 1000, 164: 1000, 101: 1000, 102: 1000, 103: 1000, 145: 1000, 170: 1000, 183: 1000, 141: 1000, 143: 1000, 112: 1000, 113: 1000, 114: 1000, 180: 1000, 181: 1000, 150: 788, 151: 1000, 190: 1000}


In [ ]:
wrong_predictions = test_ob.features_df['y_pred']!=test_ob.features_df['y_true']

In [ ]:
print(np.sum(wrong_predictions))

In [ ]:
wrong_df = test_ob.features_df[wrong_predictions]

In [ ]:
for row in wrong_df.iter:
    print(row)
    for band, i in enumerate(bands):
        pred_coeff = [0,0,0]
        for j in range(num_pc_components):
            colname = str(i)+"pc"+str(j)
            pred_coeff[i,j] = row[col_name]

In [ ]:
stopper

In [ ]:
#save_fig_base_path = 'Final_Results/'
#save_fig_postfix_path = 'case1_master_random_current_date_only_test_30_days_alert_neglect_non_estimated_curves'

In [ ]:
fig = pe.plot_performance_statistics(y_score=y_score)
fig.savefig(save_fig_base_path+"performance_stat"+save_fig_postfix_path)
#fig.savefig("randomplt")
plt.show()
plt.close('all')
#pe.plot_performance_statistics(y_score=y_score, save_fig_path=save_fig_base_path+"performance_stat"+save_fig_postfix_path)

In [ ]:
x_limits = (-5000,5000)
y_limits = (-5000,5000)
fig = train_ob.plot_features_correlation(color_band_dict=color_band_dict, x_limits=x_limits, y_limits=y_limits)
fig.savefig(save_fig_base_path+"pc_correlation"+save_fig_postfix_path)
plt.show()

In [ ]:
#print(train_df)
x_limits = (-5000,5000)
y_limits = (-5000,5000)
fig = train_ob.plot_band_correlation(color_band_dict=color_band_dict, x_limits=x_limits, y_limits=y_limits)
fig.savefig(save_fig_base_path+"band_correlation"+save_fig_postfix_path)
plt.show()